In [1]:

import glob
import os
import sys
import time
import carla

# Add CARLA Python egg path
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


In [2]:

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Destroy all actors before starting
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


In [3]:

# Load spawn points and blueprints
spawn_points = world.get_map().get_spawn_points()
blueprints = world.get_blueprint_library()

truck_bp = blueprints.filter('*firetruck*')[0]
mini_bp = blueprints.filter('*cooper_s*')[0]
vw_bp = blueprints.filter('*volkswagen*')[0]


In [5]:

# Visualize all spawn points with BIG LABEL NUMBERS
print("📍 Drawing enlarged label numbers at each spawn point...")

for idx, sp in enumerate(spawn_points):
    loc = sp.location + carla.Location(z=2.5)

    # Simulate bold/bigger text by drawing multiple overlapping strings
    offsets = [(-0.15, 0), (0.15, 0), (0, -0.15), (0, 0.15), (0, 0)]
    for dx, dy in offsets:
        offset_loc = loc + carla.Location(x=dx, y=dy)
        world.debug.draw_string(offset_loc, f"#{idx}", draw_shadow=True,
                                color=carla.Color(r=255, g=255, b=0), life_time=60.0, persistent_lines=True)

print(f"✅ {len(spawn_points)} bold label numbers displayed in simulator.")

# === Show All Navigable Spawn Points ===
print("\n--- Available Navigation Points (Index: Location) ---")
for idx, sp in enumerate(spawn_points):
    loc = sp.location
    print(f"[{idx}] x={loc.x:.2f}, y={loc.y:.2f}, z={loc.z:.2f}")

# === User Selection ===
start_index = int(input("\nEnter index for START location: "))
spoof_index = int(input("Enter index for SPOOFED GPS location: "))
end_index = int(input("Enter index for FINAL DESTINATION: "))

start_transform = spawn_points[start_index]
spoof_location = spawn_points[spoof_index].location
end_transform = spawn_points[end_index]

print("\n=== Navigation Plan Summary ===")
print(f"Start Location:      {start_transform.location}")
print(f"Spoofed GPS Location:{spoof_location}")
print(f"Destination:         {end_transform.location}")


📍 Drawing enlarged label numbers at each spawn point...
✅ 155 bold label numbers displayed in simulator.

--- Available Navigation Points (Index: Location) ---
[0] x=106.42, y=-12.71, z=0.60
[1] x=109.95, y=-17.19, z=0.60
[2] x=99.08, y=42.14, z=0.60
[3] x=106.02, y=50.87, z=0.60
[4] x=109.50, y=53.29, z=0.60
[5] x=-67.05, y=-68.69, z=0.60
[6] x=-103.22, y=-2.21, z=0.60
[7] x=109.91, y=-6.93, z=0.60
[8] x=102.93, y=-9.38, z=0.60
[9] x=-2.65, y=-68.05, z=0.60
[10] x=-24.34, y=-57.79, z=0.60
[11] x=99.38, y=-6.31, z=0.60
[12] x=98.80, y=82.89, z=0.60
[13] x=40.39, y=41.95, z=0.60
[14] x=19.35, y=137.46, z=0.60
[15] x=10.91, y=-57.40, z=0.60
[16] x=7.41, y=-60.90, z=0.60
[17] x=106.51, y=-21.55, z=0.90
[18] x=109.96, y=-27.33, z=0.60
[19] x=30.02, y=133.95, z=0.60
[20] x=-6.55, y=137.23, z=0.60
[21] x=-41.85, y=-30.44, z=0.60
[22] x=-52.07, y=82.65, z=0.60
[23] x=-28.58, y=140.54, z=0.60
[24] x=-18.39, y=130.21, z=0.60
[25] x=-9.88, y=-57.55, z=0.60
[26] x=11.18, y=-64.40, z=0.60
[27] x=-


=== Navigation Plan Summary ===
Start Location:      Location(x=-18.782679, y=-64.708092, z=0.900005)
Spoofed GPS Location:Location(x=-52.330811, y=-14.039614, z=0.600000)
Destination:         Location(x=-28.726021, y=28.104218, z=0.600000)


In [6]:

# Set spectator view
spectator = world.get_spectator()
spectator_pos = carla.Transform(start_transform.location + carla.Location(x=20, y=10, z=4),
                                carla.Rotation(yaw=start_transform.rotation.yaw - 155))
spectator.set_transform(spectator_pos)


In [7]:

# Spawn truck at start point
truck = world.try_spawn_actor(truck_bp, start_transform)
if truck:
    truck.set_autopilot(True)
    print("🚒 Firetruck spawned and set to autopilot.")


🚒 Firetruck spawned and set to autopilot.


In [8]:

# Spawn Mini at another point and move it
mini = world.try_spawn_actor(mini_bp, spawn_points[10])
if mini:
    mini_pos = carla.Transform(start_transform.location + carla.Location(x=1, z=10),
                               carla.Rotation(yaw=start_transform.rotation.yaw))
    mini.set_transform(mini_pos)
    print("🚗 Mini spawned above the ground.")


🚗 Mini spawned above the ground.


In [9]:

# Spawn VW at a new location and drop it
vw = world.try_spawn_actor(vw_bp, spawn_points[20])
if vw:
    vw_pos = carla.Transform(start_transform.location + carla.Location(x=1, z=6),
                             carla.Rotation(yaw=start_transform.rotation.yaw))
    vw.set_transform(vw_pos)
    print("🚙 VW spawned above the ground.")


🚙 VW spawned above the ground.


In [ ]:

# Print spoofed GPS info (simulated)
if truck:
    print(f"\n[Truck] Original GPS: Latitude=N/A, Longitude=N/A")
    print(f"[Truck] Spoofed GPS: Latitude={spoof_location.x}, Longitude={spoof_location.y}")



[Truck] Original GPS: Latitude=N/A, Longitude=N/A
[Truck] Spoofed GPS: Latitude=-52.330810546875, Longitude=-14.039613723754883


: 